This section aims to try different approaches. Some of the project's choices might be hindering the model's performance such as brand-model grouped imputation.

#

In [1]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [2]:
import os
import pandas as pd
import numpy as np
import json

# Load standardized paths, ml utilities and custom transformer
from project_utils.paths import (
    CLEANED_DATA_DIR,
    TEST_PREDICTIONS_DIR,
)
import project_utils.ml_utils as ml_utils

# Holdout method
from sklearn.model_selection import train_test_split
# Data pre-processing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, TargetEncoder

# Models and stacking ensemble
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    ExtraTreesRegressor,
    BaggingRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV


2025-12-20 14:02:27.622 | INFO     | project_utils.paths:<module>:19 - PROJ_ROOT path is: /Users/alexandre/Documents/GitHub/price_predictor


In [3]:
df_train = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"), index_col="carID"
)
X_test = pd.read_csv(
    os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"), index_col="carID"
)

X = df_train.drop("price", axis=1).copy()
y = df_train["price"].copy()

# Performing the holdout methods
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)

# Experimenting new pipeline

In this notebook, the analysis focuses exclusively on decision tree–based models, which are expected to provide strong predictive performance for this task.

+ IterativeImputer - Iterative imputation was employed as a strategy for handling missing numerical values by modelling each feature with missing entries as a function of the remaining features. Given the presence of multiple multivariate relationships within the dataset, this approach has the potential to yield more accurate imputations than univariate methods.

+ SimpleImputer - For categorical variables, missing values were imputed using a simple imputation strategy based on the most frequent category per feature. This method preserves the modal structure of categorical variables while avoiding the introduction of artificial categories.

+ StandardScaler - As several highly skewed numerical features were previously log-transformed, a z-score standardisation approach was adopted for feature scaling. This method centres features to zero mean and unit variance, facilitating stable optimisation and comparability across numerical variables.

It should also be noted that, within this pipeline, categorical features were not scaled. This design choice is justified by the focus of this notebook on tree-based models, which are invariant to feature scaling. In contrast, scaling was applied after target encoding in earlier pipelines, as failing to scale categorical features transformed into continuous variables can adversely affect the performance of scale-sensitive models, such as linear and instance-based regressors.

In [4]:
# --- Defining numerical and categorical feature columns --- #
numeric_cols = X_train.select_dtypes(exclude="object").columns.to_list()
categorical_cols = X_train.select_dtypes(include="object").columns.to_list()

Features identified for removal through filter-based methods were excluded from subsequent modelling stages, with the exception of those discarded by the Elastic Net regressor. These features were retained, as Elastic Net’s linear and additive nature may suppress variables that are weak in isolation but informative when combined or exploited by non-linear models. Consequently, preserving these features allows downstream models to capture complex relationships that Elastic Net may fail to represent.

In [5]:
pre_processor = Pipeline(
    steps=[
        (
            "column_transformer",
            ColumnTransformer(
                transformers=[
                    # --- Numeric pipeline ---
                    (
                        "num",
                        Pipeline(
                            steps=[
                                (
                                    "imputer",
                                    IterativeImputer(
                                        estimator=None,
                                        n_nearest_features=5,
                                        random_state=0,
                                    ),
                                ),
                                ("scaler", StandardScaler()),
                            ]
                        ),
                        numeric_cols,
                    ),
                    # --- Categorical pipeline ---
                    (
                        "cat",
                        Pipeline(
                            steps=[
                                ("imputer", SimpleImputer(strategy="most_frequent")),
                                (
                                    "encoder",
                                    TargetEncoder(
                                        categories="auto",
                                        target_type="continuous",
                                        smooth="auto",
                                    ),
                                ),
                            ]
                        ),
                        categorical_cols,
                    ),
                ]
            ),
        )
    ]
)


The same parameter grid and randomized search configuration were applied to the Histogram Gradient Boosting Regressor in this notebook to perform hyperparameter optimisation.


In [6]:
# Import JSON file with best_params selected from the RandomSearch
RANDOM_SEARCH = os.path.join(PROJ_ROOT, "src", "models", "random_search")
with open(os.path.join(RANDOM_SEARCH, "open_ended_params.json"), "r") as f:
    best_params = json.load(f)

clean_params = {
    k.replace("regressor__regressor__", ""): v for k, v in best_params.items()
}

# --- HGBR with log-transform target --- #
log_tree_regressor = TransformedTargetRegressor(
    regressor=HistGradientBoostingRegressor(**clean_params),
    func=np.log1p,
    inverse_func=np.expm1,
)

model = Pipeline(
    [
        ("preprocessor", pre_processor),
        ("regressor", log_tree_regressor),
    ]
)

hgbr_model = model.fit(X_train, y_train)
hgbr_model

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} 

In [7]:
# Compute evaluation metrics for the training and validation sets
# Training set
train_predictions = model.predict(X_train)
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(
    y_train, train_predictions, verbose=True
)
# Validation set
validation_predictions = model.predict(X_val)
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(
    y_val, validation_predictions, verbose=True
)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training set metrics:
R²: 0.936, MAE: 1182.51, MAPE: 7.10%, RMSE: 2457.17
-------------------------------------------------------
Validation set metrics:
R²: 0.915, MAE: 1465.69, MAPE: 8.58%, RMSE: 2959.97
-------------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

In [9]:
# --- ETR with log-transform target --- #
extr = ExtraTreesRegressor(
    criterion="squared_error",
    random_state=0,
    bootstrap=True,
    n_jobs=-1,
)

log_extr = TransformedTargetRegressor(
    regressor=extr,
    func=np.log1p,
    inverse_func=np.expm1,
)

pipeline = Pipeline(
    [
        ("preprocessor", pre_processor),
        ("regressor", log_extr),
    ]
)

etr_model = pipeline.fit(X_train, y_train)
etr_model

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('regressor', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} 

In [10]:
# Compute evaluation metrics for the training and validation sets
# Training set
train_predictions = pipeline.predict(X_train)
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(
    y_train, train_predictions, verbose=True
)
# Validation set
validation_predictions = pipeline.predict(X_val)
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(
    y_val, validation_predictions, verbose=True
)


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training set metrics:
R²: 0.962, MAE: 968.85, MAPE: 5.78%, RMSE: 1892.88
-------------------------------------------------------
Validation set metrics:
R²: 0.919, MAE: 1475.19, MAPE: 8.67%, RMSE: 2892.92
-------------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

In [11]:
# --- Bagging regressor (tree bagging; good diversity for stacking) ---
bagger = BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=0),
    n_estimators=300,
    max_samples=0.8,
    bootstrap=True,
    random_state=0,
    n_jobs=-1,
)

log_bagger = TransformedTargetRegressor(
    regressor=bagger,
    func=np.log1p,
    inverse_func=np.expm1,
)

bagging_model = Pipeline(
    steps=[
        ("preprocessor", pre_processor),
        ("regressor", log_bagger),
    ]
)
bg_model = bagging_model.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

In [12]:
# Compute evaluation metrics for the training and validation sets
# Training set
train_predictions = bagging_model.predict(X_train)
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(
    y_train, train_predictions, verbose=True
)
# Validation set
validation_predictions = bagging_model.predict(X_val)
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(
    y_val, validation_predictions, verbose=True
)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training set metrics:
R²: 0.956, MAE: 996.97, MAPE: 5.93%, RMSE: 2037.91
-------------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Validation set metrics:
R²: 0.926, MAE: 1464.80, MAPE: 8.63%, RMSE: 2760.54
-------------------------------------------------------


In [15]:
estimators = [
    ("hgbr", hgbr_model),
    ("etr", etr_model),
    ("bg", bg_model),
]

# --- RidgeCV meta-learner ---
ridge_meta = RidgeCV(alphas=np.logspace(-4, 4, 20), cv=10)

stacking_ensemble = StackingRegressor(
    estimators=estimators,
    final_estimator=ridge_meta,
    cv=10,
    n_jobs=-1,
)

# Optional: log-transform the final ensemble target too
reg = TransformedTargetRegressor(
    regressor=stacking_ensemble,
    func=np.log1p,
    inverse_func=np.expm1,
)

reg.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

,"regressor regressor: object, default=NoneRegressor object such as derived from:class:`~sklearn.base.RegressorMixin`. This regressor willautomatically be cloned each time prior to fitting. If `regressor isNone`, :class:`~sklearn.linear_model.LinearRegression` is created and used.",StackingRegre... n_jobs=-1)
,"transformer transformer: object, default=NoneEstimator object such as derived from:class:`~sklearn.base.TransformerMixin`. Cannot be set at the same timeas `func` and `inverse_func`. If `transformer is None` as well as`func` and `inverse_func`, the transformer will be an identitytransformer. Note that the transformer will be cloned during fitting.Also, the transformer is restricting `y` to be a numpy array.",None
,"func func: function, default=NoneFunction to apply to `y` before passing to :meth:`fit`. Cannot be setat the same time as `transformer`. If `func is None`, the function used will bethe identity function. If `func` is set, `inverse_func` also needs to beprovided. The function needs to return a 2-dimensional array.",<ufunc 'log1p'>
,"inverse_func inverse_func: function, default=NoneFunction to apply to the prediction of the regressor. Cannot be set atthe same time as `transformer`. The inverse function is used to returnpredictions to the same space of the original training labels. If`inverse_func` is set, `func` also needs to be provided. The inversefunction needs to return a 2-dimensional array.",<ufunc 'expm1'>
,"check_inverse check_inverse: bool, default=TrueWhether to check that `transform` followed by `inverse_transform`or `func` followed by `inverse_func` leads to the original targets.",True
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('column_transformer', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them thro

In [ ]:
# Compute evaluation metrics for the training and validation sets
# Training set
train_predictions = reg.predict(X_train)
print("Training set metrics:")
metrics_train = ml_utils.linear_evaluation_metrics(
    y_train, train_predictions, verbose=True
)
# Validation set
validation_predictions = reg.predict(X_val)
print("Validation set metrics:")
metrics_val = ml_utils.linear_evaluation_metrics(
    y_val, validation_predictions, verbose=True
)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma

Training set metrics:
R²: 0.955, MAE: 1112.99, MAPE: 6.62%, RMSE: 2050.48
-------------------------------------------------------
Validation set metrics:
R²: 0.934, MAE: 1427.42, MAPE: 8.41%, RMSE: 2609.56
-------------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


This approach results in a substantial improvement in model performance. Several methodological strategies introduced in this section contribute positively to predictive accuracy, as evidenced by a marked reduction across all evaluation metrics. Furthermore, the close alignment between training and validation results suggests that the model generalises well and does not exhibit signs of overfitting. Overall, this exploratory section demonstrates how the thoughtful adjustment of modelling strategies can lead to meaningful gains in predictive performance.

In [ ]:
# -- Export X_test predictions --#
pred_y_test = reg.predict(X_test)
# Convert to Series
test_predictions_series = pd.Series(data=pred_y_test, name="price", index=X_test.index)

# Export CSV file
test_predictions_series.to_csv(
    os.path.join(TEST_PREDICTIONS_DIR, "open_ended_ensemble_test_predictions.csv")
)

/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/homebrew/Caskroom/miniconda/base/envs/car_price_env/lib/python3.12/site-packages/sklearn/linear_model/_base.py:293: RuntimeWarning: overflow encountered in ma